<a href="https://colab.research.google.com/github/Afique1/Business-Review-by-user/blob/Alan's-works/SIT742Task2_Group32.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Questions selection
Questions are selected based on the birthdate(day) of students [if it's even/odd then selected Q. are even/odd]

In [ ]:
students = {"Afique": 11,"Alan": 10}
questions = list(range(1, 9))  # 8 questions for Part 1

assignments = {}

for student, birthdate in students.items():
  assignments[student] = []
  for question in questions:
    if (birthdate % 2 == 0 and question % 2 == 0) or (birthdate % 2 != 0 and question % 2 != 0):
      assignments[student].append(f"Q{question}")

print("Assigned Questions for part 1:\n"+"-"*40)

for student, assigned_questions in assignments.items():
  print(f"{student} (Birthdate: {students[student]}): {', '.join(assigned_questions)}")

Assigned Questions for part 1:
----------------------------------------
Afique (Birthdate: 11): Q1, Q3, Q5, Q7
Alan (Birthdate: 10): Q2, Q4, Q6, Q8


# Part 0) Read the data using Pyspark

Update and Import packages

In [ ]:
#update local version of the package catalog
!apt-get update
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install findspark
!pip install -q findspark

import findspark
findspark.init()

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://cli.github.com/packages stable/main amd64 Packages [346 B]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,006 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ub

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# from pyspark import SparkContext
# from pyspark.sql import SQLContext

`SparkSession` is the combination of SparkContext and SQLContext. So, the updated package/ will be used instead of SparkContext and SQLContext.


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("BusinessReviewAnalysis") \
    .getOrCreate()

print("Spark session created successfully!")

Spark session created successfully!


Download and save the csv file locally.

In [6]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=4ad8942648319396f8c4cbf178a3488cc3e5756b810d606b377c2edcae6107c3
  Stored in directory: /root/.cache/pip/wheels/01/46/3b/e29ffbe4ebe614ff224bad40fc6a5773a67a163251585a13a9
Successfully built wget


In [7]:
import wget

link_to_data1 = 'https://github.com/Afique1/Business-Review-by-user/raw/main/Dataset/meta-review-business.csv'
link_to_data2 = 'https://github.com/Afique1/Business-Review-by-user/raw/main/Dataset/review.csv'

MetaReview_Dataset = wget.download(link_to_data1)
Review_Dataset = wget.download(link_to_data2)

In [9]:
# Read the datasets into Spark DataFrames with 10 partitioning
meta_review_sdf = spark.read.csv(MetaReview_Dataset, header=True, inferSchema=True).repartition(10)
review_sdf = spark.read.csv(Review_Dataset, header=True, inferSchema=True).repartition(10)

print("Meta Review Dataset Schema:")
meta_review_sdf.printSchema()
# print("\nMeta Review Dataset Head:")
# meta_review_sdf.show(5)

print("\nReview Dataset Schema:")
review_sdf.printSchema()
# print("\nReview Dataset Head:")
# review_sdf.show(5)

Meta Review Dataset Schema:
root
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- gmap_id: string (nullable = true)
 |-- description: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- category: string (nullable = true)
 |-- avg_rating: string (nullable = true)
 |-- num_of_reviews: string (nullable = true)
 |-- price: string (nullable = true)
 |-- hours: string (nullable = true)
 |-- MISC: string (nullable = true)
 |-- state: string (nullable = true)
 |-- relative_results: string (nullable = true)
 |-- url: string (nullable = true)


Review Dataset Schema:
root
 |-- user_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- time: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- text: string (nullable = true)
 |-- pics: string (nullable = true)
 |-- resp: string (nullable = true)
 |-- gmap_id: string (nullable = true)



Investigate the datasets

In [10]:
print("Investigating Meta Review Dataset:")
print("Number of rows:", meta_review_sdf.count())
print("Columns:", meta_review_sdf.columns)
print("Summary Statistics:")
meta_review_sdf.describe().show()

print("\nInvestigating Review Dataset:")
print("Number of rows:", review_sdf.count())
print("Columns:", review_sdf.columns)
print("Summary Statistics:")
review_sdf.describe().show()

Investigating Meta Review Dataset:
Number of rows: 12774
Columns: ['name', 'address', 'gmap_id', 'description', 'latitude', 'longitude', 'category', 'avg_rating', 'num_of_reviews', 'price', 'hours', 'MISC', 'state', 'relative_results', 'url']
Summary Statistics:
+-------+--------------------+--------------------+--------------------+--------------------+------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|summary|                name|             address|             gmap_id|         description|          latitude|          longitude|            category|          avg_rating|      num_of_reviews|               price|               hours|                MISC|               state|    relative_results|                 url|
+-------+--------------------+--------------------+--------------------+--------------------+

In [ ]:
print("Number of partitions for meta_review_sdf:", meta_review_sdf.rdd.getNumPartitions())
print("Number of partitions for review_sdf:", review_sdf.rdd.getNumPartitions())

Number of partitions for meta_review_sdf: 10
Number of partitions for review_sdf: 10


In [11]:
# Get the SparkContext from the SparkSession
sc = spark.sparkContext

# Get the number of active cores
# In local mode, this typically corresponds to the number of threads being used
num_cores = sc.defaultParallelism

print(f"Number of cores being used by Spark: {num_cores}")

Number of cores being used by Spark: 2


Is it wise to join both sdfs?

In [12]:
# Count and print the number of unique gmap_id values in each DataFrame
unique_meta_review_count = meta_review_sdf.select("gmap_id").distinct().count()
unique_review_count = review_sdf.select("gmap_id").distinct().count()

print("\nNumber of unique gmap_id in meta_review_sdf:", unique_meta_review_count)
print("Number of unique gmap_id in review_sdf:", unique_review_count)


Number of unique gmap_id in meta_review_sdf: 12688
Number of unique gmap_id in review_sdf: 12468


## Join based on `gmap_id`

In [13]:
# Perform an inner join on the 'gmap_id' column
sdf = meta_review_sdf.join(review_sdf, on="gmap_id", how="inner")

# Show the schema and a few rows of the joined DataFrame to verify
print("Joined DataFrame Schema:")
sdf.printSchema()
sdf.count()

Joined DataFrame Schema:
root
 |-- gmap_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- description: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- category: string (nullable = true)
 |-- avg_rating: string (nullable = true)
 |-- num_of_reviews: string (nullable = true)
 |-- price: string (nullable = true)
 |-- hours: string (nullable = true)
 |-- MISC: string (nullable = true)
 |-- state: string (nullable = true)
 |-- relative_results: string (nullable = true)
 |-- url: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- time: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- text: string (nullable = true)
 |-- pics: string (nullable = true)
 |-- resp: string (nullable = true)



498903

In [14]:
from pyspark.sql.functions import col

# Sort meta_review_sdf by gmap_id and show the top 5
print("Top 5 gmap_id in meta_review_sdf:")
meta_review_sdf.select("gmap_id").orderBy(col("gmap_id")).show(5)

# Sort review_sdf by gmap_id and show the top 5
print("\nTop 5 gmap_id in review_sdf:")
review_sdf.select("gmap_id").orderBy(col("gmap_id")).show(5)

Top 5 gmap_id in meta_review_sdf:
+--------------------+
|             gmap_id|
+--------------------+
|      1700 Willow St|
|              2nd St|
| 8225 Old Seward ...|
|        8600 King St|
|        8600 King St|
+--------------------+
only showing top 5 rows


Top 5 gmap_id in review_sdf:
+-------+
|gmap_id|
+-------+
|   NULL|
|   NULL|
|   NULL|
|   NULL|
|   NULL|
+-------+
only showing top 5 rows



# Part 1
---



## Q1.1 Some Data Wrangling

### Q1.1.1 none, null in `text` column to "no review"

### Q1.1.2 Change the format of time
in the `time` column, convert the strings from time to `yyyy-mm-dd` format
in the new column as `newtime` and show the first 5 rows.

## Q1.2 Info for `gmap_id` on the `review`

### Q1.2.1 using pyspark, calculate the number of reviews for each unique `gmap_id`
(save as float format in pyspark dataframe to show the top 5 rows)

In [ ]:
from pyspark.sql.functions import sum as spark_sum
from pyspark.sql.functions import col

Calculation based on the Review Dataset review_sdf

In [60]:
review_sdf_counts_spark = review_sdf.groupBy("gmap_id").count().withColumn("count", col("count").cast("float"))

In [61]:
review_sdf_counts_spark.orderBy(col("count").desc()).show(5)

+--------------------+-------+
|             gmap_id|  count|
+--------------------+-------+
|                NULL|46309.0|
|0x56c897b9ce6000d...| 2737.0|
|0x56c899d05892048...| 2555.0|
|0x56c897c63697ee3...| 2220.0|
|0x56c8965ee2fb87a...| 2202.0|
+--------------------+-------+
only showing top 5 rows



In [69]:
# Cross-checking the counting results
review_sdf_counts_spark.agg(spark_sum("count")).collect()[0][0] # expected to match the no. of rows as 552624

552624.0

Calculation based on the joined datafram sdf

In [66]:
sdf_counts_spark = sdf.groupBy("gmap_id").count().withColumn("count", col("count").cast("float"))

In [67]:
sdf_counts_spark.orderBy(col("count").desc()).show(5)

+--------------------+------+
|             gmap_id| count|
+--------------------+------+
|0x56c897b9ce6000d...|2737.0|
|0x56c899d05892048...|2555.0|
|0x56c897c63697ee3...|2220.0|
|0x56c8965ee2fb87a...|2202.0|
|0x56c89629bde7481...|2156.0|
+--------------------+------+
only showing top 5 rows



In [68]:
sdf_counts_spark.agg(spark_sum("count")).collect()[0][0]

498903.0

In [49]:
# A suspected phantom row indicated by 1 counted in gmap_id of "Anchorage"
sdf_counts_spark.orderBy(col("count")).show(5)

+--------------------+-----+
|             gmap_id|count|
+--------------------+-----+
|           Anchorage|  1.0|
|0x56c897c88b2e4b7...|  4.0|
|0x56c8962f33efc3b...|  5.0|
|0x56c8eb715388aae...|  5.0|
|0x5400df8600be380...|  5.0|
+--------------------+-----+
only showing top 5 rows



In [52]:
# The phantom row has been identified yet has no impact
sdf.filter(col("gmap_id") == "Anchorage").show()

sdf_phantom_cleaned = sdf.filter(col("gmap_id") != "Anchorage")
sdf_phantom_cleaned_counts_spark = sdf_phantom_cleaned.groupBy("gmap_id").count().withColumn("count", col("count").cast("float"))
sdf_phantom_cleaned_counts_spark.agg(spark_sum("count")).collect()[0][0]

+-------+----+-------+-----------+--------+---------+--------+----------+--------------+-----+-----+----+-----+----------------+---+-------+----+----+------+----+----+----+
|gmap_id|name|address|description|latitude|longitude|category|avg_rating|num_of_reviews|price|hours|MISC|state|relative_results|url|user_id|name|time|rating|text|pics|resp|
+-------+----+-------+-----------+--------+---------+--------+----------+--------------+-----+-----+----+-----+----------------+---+-------+----+----+------+----+----+----+
+-------+----+-------+-----------+--------+---------+--------+----------+--------------+-----+-----+----+-----+----------------+---+-------+----+----+------+----+----+----+



498903.0

### Q1.2.2 convert the pyspark df to pandas df
* create a new column `review_time` with the info of review time on hours level, add in the pandas df.
* show top 5 rows

In [189]:
df = review_sdf.toPandas() # transform the current pyspark dataframe to pandas dataframe

In [190]:
df["time"].head(10)

,time
0,1525676908252
1,1368006776281
2,1569090797829
3,1597542582178
4,1598037847651
5,None
6,1549419589149
7,"""{'time': 1598807097467, 'text': """"We all appr..."
8,1495309866441
9,1562915528083


In [211]:
import re

def clean_and_convert(val):
    try:
        if pd.isnull(val):
            return pd.NaT

        # Directly handle numeric values
        if isinstance(val, (int, float)):
            return pd.to_datetime(int(val), unit="ms")

        # Handle dictionary-like objects
        if isinstance(val, dict) and "time" in val:
            return pd.to_datetime(int(val["time"]), unit="ms")

        # Convert to string and strip whitespace
        val_str = str(val).strip()

        # Try to extract timestamp from dictionary-like string
        match = re.search(r"[\"']?time[\"']?\s*:\s*(\d{13})", val_str)
        if match:
            return pd.to_datetime(int(match.group(1)), unit="ms")

        # Fallback: check if the string itself is a 13-digit number
        if re.fullmatch(r"\d{13}", val_str):
            return pd.to_datetime(int(val_str), unit="ms")

    except Exception:
        pass

    return pd.NaT

In [224]:
df["review_time"] = df["time"].apply(clean_and_convert) # create the column reivew_time with the information of review time

In [225]:
df[["time", "review_time"]].sample(20)

,time,review_time
224260,1483482680346,2017-01-03 22:31:20.346
184957,1544800866752,2018-12-14 15:21:06.752
48684,1573280756845,2019-11-09 06:25:56.845
294193,1572670298088,2019-11-02 04:51:38.088
518510,1614574763005,2021-03-01 04:59:23.005
447366,1478381835908,2016-11-05 21:37:15.908
495764,1487806991963,2017-02-22 23:43:11.963
500861,1567540780666,2019-09-03 19:59:40.666
405196,1530945605693,2018-07-07 06:40:05.693
542574,1609394663329,2020-12-31 06:04:23.329


In [226]:
invalid_rows = df[df["review_time"].isna()]
print(invalid_rows[["time", "review_time"]])

        time review_time
5       None         NaT
22      None         NaT
46      None         NaT
49      None         NaT
63      None         NaT
...      ...         ...
552562  None         NaT
552580  None         NaT
552598  None         NaT
552605  None         NaT
552623  None         NaT

[29897 rows x 2 columns]


In [227]:
def contains_timestamp(val):
    if isinstance(val, str):
        match = re.search(r"\b\d{13}\b", val)
        return bool(match)
    return False

In [228]:
print(invalid_rows[~invalid_rows["time"].isna()]["time"])
print()

mask = invalid_rows["time"].notnull() & invalid_rows["time"].apply(contains_timestamp)
shortlist = invalid_rows[mask]

print(shortlist[["time"]]) # emptiness means no more entries with extractable 13-digit timestamps

225        super soggy bread and the cheese choice was b...
276        could probably spend at least a week eating h...
518        asking me to move it to 5. I rearranged my sc...
837        expecting a call but nothing. I called back a...
924        I took her for a spin and it drove like a new...
                                ...                        
551665     because they have a Specials menu that is...w...
551771                                     Xander and Tina.
551842                there is no way not to give 5 stars."
552280                                  good house fries...
552521     and everything was small to accommodate all c...
Name: time, Length: 5082, dtype: object

Empty DataFrame
Columns: [time]
Index: []


In [230]:
df["review_time"] = df["review_time"].dt.hour

In [238]:
df[["time", "review_time"]].head(10)

,time,review_time
0,1525676908252,7.0
1,1368006776281,9.0
2,1569090797829,18.0
3,1597542582178,1.0
4,1598037847651,19.0
5,None,NaN
6,1549419589149,2.0
7,"""{'time': 1598807097467, 'text': """"We all appr...",17.0
8,1495309866441,19.0
9,1562915528083,7.0


### Q1.2.3 Relationship between `gmap_id` and `review_time` [both discrete]
* which time people usually review?
* which time do they review the less amount?
* How many business is reviewed in the morning? [6<= x <12 ; 06:00-11:59]

## Q1.3 join `meta-business` with the current df on `gmap_id`

### Q1.3.1 Find the `workday` with most reviews

### Q1.3.2 Category of business name with most review in that day
* in that day which business (the business `name`) has the highest avg rating [max(avg(most_review_day))]
* Find the `category` of the business `name`

### Q1.3.3 Exploration on business `name`
* peak hour of that catefory
* Plot historam

[write about the insight]

## Q1.4 Top 30 words by year
* For the reviews on each of the submissions, work on all the review content and find out the `top 30`
most common words;
* Also generate separate `word cloud` visualizations for `different years`( by grouping
the reviews by review year )
* write down the insights in the markdown cell.

## Q1.5 Analysis on business `name` and `review`
* find the number of unique business `reviewers` and its `category` (find out the highest distinct count of reviewers on business / category level)
* When `reviewers` submitted their review? [add visualization and write in markdown]

## Q1.6 Build recommendation for business
(by using reviews, ratings, and its categories)

### Q1.6.1 Write down the strategy to build Recommendation
create own strategy or leverage KNN on collaborative filtering.
* Write about the logic

### Q1.6.2 Implement the strategy
add sufficient comments where required

## Q1.7 Analysis on `Rating`s for each submission review

### Q1.7.1 Relationship of `rating` and `business categories`

write down the insights

### Q1.7.2 Focus on lower ratings
[Select a threshold to considered as lower]
* Find the most common words
* Whats the reason behind lower rating?

## Q1.8 Reviewer level with all the reviewed business history (submission of the review)

### Q1.8.1 Create a `user_business_list`
* check reviewer level for reviewed business
* sort the review based on review time (`new_time` column)
* save the business name, for each reviewer into the `user_business_list`
* for the list: print the `number of elements` / `number of business name` for each reviewers **[before removing]**

### Q1.8.2 Repeated business names for the same user/reviewer?
* remove those duplicated business names under same user (take unique business name)
* print the `number of business name` for each reviewers **[after removing]**

### Q1.8.3 user similarities in the past reviewed business
* Write down the strategy for implementation with sufficient explanation
* Check on the `user_business_list` and find the user similarities according to their past reviewed business.

Hint: you might consider to use `encoding` for each of the business names and then calculate the difference of the users.

# Part 2
---

## Q2.1 Seasonality Pattern

## Q2.2 ARIMA Model

## Q2.3 Analyze Annual Report of Indigenous Strategy
link to the pdf

### Data Extraction

### Data Analysis

### Insights